# DeepFake CNN Classifier
This notebook trains a deepfake image classifier using PyTorch.

## Import Dependencies

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torchvision.models as models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

## Set Random Seed and Define Device

In [2]:
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


## Define Dataset Paths

In [3]:
REAL_DIR = "wiki"
FAKE_DIRS = ["inpainting", "insight", "text2img"]


## Define Dataset Class

In [4]:
class DeepFakeDataset(Dataset):
    def __init__(self, real_dir, fake_dirs, split='train', transform=None, val_split=0.15, test_split=0.15, max_images_per_class=None):
        """
        Dataset for loading DeepFake Face images.

        Args:
            real_dir: Directory containing real images
            fake_dirs: List of directories containing fake images
            split: One of 'train', 'val', or 'test'
            transform: Image transformations
            val_split: Proportion of data for validation
            test_split: Proportion of data for testing
            max_images_per_class: Maximum number of images to use per class (real/fake)
        """
        self.transform = transform

        # Get all image paths with limit
        real_images = self._get_image_paths(real_dir, label=0, max_images=max_images_per_class)  # 0 = real

        fake_images = []
        # If we have multiple fake directories, distribute the limit among them
        fake_dir_limit = None
        if max_images_per_class is not None:
            fake_dir_limit = max_images_per_class // len(fake_dirs)

        for fake_dir in fake_dirs:
            fake_images.extend(self._get_image_paths(fake_dir, label=1, max_images=fake_dir_limit))  # 1 = fake

        # Combine and shuffle
        all_images = real_images + fake_images
        np.random.shuffle(all_images)

        # Print dataset stats
        print(f"Loaded {len(real_images)} real images and {len(fake_images)} fake images")

        # Split into train, val, test
        total_size = len(all_images)
        test_size = int(total_size * test_split)
        val_size = int(total_size * val_split)
        train_size = total_size - test_size - val_size

        if split == 'train':
            self.images = all_images[:train_size]
        elif split == 'val':
            self.images = all_images[train_size:train_size+val_size]
        elif split == 'test':
            self.images = all_images[train_size+val_size:]
        else:
            raise ValueError("Split must be one of 'train', 'val', or 'test'")

        print(f"{split} dataset size: {len(self.images)}")
    
    def _get_image_paths(self, directory, label, max_images=None):
        """Get all image paths with labels from directory with an optional limit"""
        image_paths = []

        # Recursively walk through directory structure
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append((os.path.join(root, file), label))
                    # Check if we've reached the limit
                    if max_images is not None and len(image_paths) >= max_images:
                        return image_paths

        return image_paths
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        
        # Load and convert image
        try:
            image = Image.open(img_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a placeholder in case of error
            if self.transform:
                return torch.zeros((3, 224, 224)), label
            return torch.zeros((3, 224, 224)), label

## Define the CNN Model

In [5]:
class DeepFakeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(DeepFakeClassifier, self).__init__()
        
        # Use a pre-trained ResNet as base model
        self.base_model = models.resnet50(pretrained=True)
        
        # Replace final fully connected layer
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        return self.base_model(x)

## Define Training Function

In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    model.to(device)
    best_val_accuracy = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images = images.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward + backward + optimize
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = accuracy_score(all_labels, all_preds)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_all_preds = []
        val_all_labels = []
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_all_preds.extend(preds.cpu().numpy())
                val_all_labels.extend(labels.cpu().numpy())
        
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = accuracy_score(val_all_labels, val_all_preds)
        
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)
        
        # Step the scheduler
        scheduler.step(val_epoch_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")
        
        # Save the best model
        if val_epoch_acc > best_val_accuracy:
            best_val_accuracy = val_epoch_acc
            torch.save(model.state_dict(), 'best_deepfake_classifier.pth')
            print(f"Saved best model with validation accuracy: {best_val_accuracy:.4f}")
    
    return model, history

# Evaluation function


In [7]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Real', 'Fake'], 
                yticklabels=['Real', 'Fake'])
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Plot ROC curve for binary classification
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    
    # Plot training history
    def plot_history(history):
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Training Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title('Loss over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Training Accuracy')
        plt.plot(history['val_acc'], label='Validation Accuracy')
        plt.title('Accuracy over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.close()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'true_labels': all_labels,
        'probabilities': all_probs
    }

## Train and Evaluate the Model

In [8]:
def debug_main():
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create datasets (with smaller splits for testing)
    train_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='train', transform=transform, 
                                   val_split=0.05, test_split=0.05)
    val_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='val', transform=transform,
                                 val_split=0.05, test_split=0.05)
    
    # Create dataloaders with smaller batch size and fewer workers
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)
    
    # Try to process a single batch to verify everything works
    print("Testing a single batch processing...")
    model = DeepFakeClassifier(num_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Try one training iteration
    for images, labels in train_loader:
        print(f"Loaded batch: images shape {images.shape}, labels shape {labels.shape}")
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        print(f"Forward pass successful, loss: {loss.item()}")
        break
    
    print("Debug complete!")

# Run the debug function
debug_main()

Loaded 30000 real images and 90000 fake images
train dataset size: 108000
Loaded 30000 real images and 90000 fake images
val dataset size: 6000
Testing a single batch processing...


C:\Anaconda\envs\trabalho_labsiacd\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Anaconda\envs\trabalho_labsiacd\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded batch: images shape torch.Size([4, 3, 224, 224]), labels shape torch.Size([4])
Forward pass successful, loss: 0.6693949699401855
Debug complete!


In [ ]:
# este é o main original mas nao consegui corrê-lo porque demorava muito tempo
def main():
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create datasets
    train_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='train', transform=transform)
    val_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='val', transform=transform)
    test_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='test', transform=transform)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    # Create model
    model = DeepFakeClassifier(num_classes=2)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Train model
    model, history = train_model(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        criterion=criterion, 
        optimizer=optimizer,
        scheduler=scheduler, 
        num_epochs=15
    )
    
    # Load best model for evaluation
    model.load_state_dict(torch.load('best_deepfake_classifier.pth'))
    
    # Evaluate model
    evaluation_results = evaluate_model(model, test_loader)
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Training Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    print("Training and evaluation complete!")

if __name__ == "__main__":
    main()


Loaded 30000 real images and 90000 fake images
train dataset size: 84000
Loaded 30000 real images and 90000 fake images
val dataset size: 18000
Loaded 30000 real images and 90000 fake images
test dataset size: 18000


C:\Anaconda\envs\trabalho_labsiacd\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/15 - Training:   0%|                                                                  | 0/2625 [00:00<?, ?it/s]

In [9]:
def main():
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Set a limit for the number of images per class
    max_images_per_class = 500  # You can adjust this number
    
    # Create datasets with limited images
    train_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='train', transform=transform, max_images_per_class=max_images_per_class)
    val_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='val', transform=transform, max_images_per_class=max_images_per_class)
    test_dataset = DeepFakeDataset(REAL_DIR, FAKE_DIRS, split='test', transform=transform, max_images_per_class=max_images_per_class)
    
    # Create dataloaders with smaller batch size
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)
    
    # Create model
    model = DeepFakeClassifier(num_classes=2)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Train model with fewer epochs
    model, history = train_model(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        criterion=criterion, 
        optimizer=optimizer,
        scheduler=scheduler, 
        num_epochs=5  # Reduced epochs for faster training
    )
    
    # Load best model for evaluation
    model.load_state_dict(torch.load('best_deepfake_classifier.pth'))
    
    # Evaluate model
    evaluation_results = evaluate_model(model, test_loader)
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Training Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    print("Training and evaluation complete!")

# Run main with limited dataset
main()

Loaded 500 real images and 498 fake images
train dataset size: 700
Loaded 500 real images and 498 fake images
val dataset size: 149
Loaded 500 real images and 498 fake images
test dataset size: 149


C:\Anaconda\envs\trabalho_labsiacd\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/5 - Validation: 100%|██████████████████████████████████████████████████████████| 19/19 [00:22<00:00,  1.19s/it]


Epoch 1/5 - Train Loss: 0.8338, Train Acc: 0.5000, Val Loss: 0.6937, Val Acc: 0.5705
Saved best model with validation accuracy: 0.5705


Epoch 2/5 - Validation: 100%|██████████████████████████████████████████████████████████| 19/19 [00:20<00:00,  1.09s/it]


Epoch 2/5 - Train Loss: 0.7088, Train Acc: 0.4686, Val Loss: 0.6928, Val Acc: 0.5101


...

Epoch 4/5 - Train Loss: 0.7004, Train Acc: 0.5086, Val Loss: 0.6947, Val Acc: 0.4899


Epoch 5/5 - Validation: 100%|██████████████████████████████████████████████████████████| 19/19 [00:20<00:00,  1.07s/it]


Epoch 5/5 - Train Loss: 0.6987, Train Acc: 0.4914, Val Loss: 0.7200, Val Acc: 0.4899


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:21<00:00,  1.14s/it]


Test Accuracy: 0.5369
Precision: 0.5373
Recall: 0.5369
F1 Score: 0.5341
Training and evaluation complete!
